# Byte Pair Encoding (BPE)

**Descrição:**
Este notebook apresenta o funcionamento do **Byte Pair Encoding (BPE)**, um método de tokenização baseado em subpalavras amplamente utilizado em modelos de linguagem modernos. Diferentemente da tokenização por palavras, o BPE constrói tokens a partir de **padrões frequentes de caracteres**, permitindo representar palavras raras ou desconhecidas por meio da combinação de unidades menores.

**Objetivo:**
Demonstrar, de forma didática e passo a passo, como o algoritmo de Byte Pair Encoding:

* Aprende um vocabulário de subpalavras a partir de um corpus
* Reduz o problema de palavras fora do vocabulário (OOV)
* Cria uma representação compacta e eficiente do texto
* Converte texto em sequências de tokens subword (encode)
* Reconstrói o texto a partir desses tokens (decode)

O objetivo é entender por que o BPE é uma peça fundamental na preparação de texto para o treinamento de LLMs.

**Funcionamento:**

![Byte pair encoder](../../imagens/04_byte_pair_encode.png)

O funcionamento do BPE segue um processo iterativo de aprendizado e aplicação:

1. **Inicialização do vocabulário:**
   O texto do corpus é inicialmente representado como sequências de caracteres individuais, geralmente com um marcador de fim de palavra.

2. **Aprendizado por fusões (merges):**
   O algoritmo identifica o par de símbolos adjacentes mais frequente no corpus e os funde em um novo token.
   Esse processo é repetido iterativamente, expandindo gradualmente o vocabulário com subpalavras mais frequentes.

3. **Encode (tokenização):**
   Um novo texto é decomposto em subpalavras usando as regras de fusão aprendidas, garantindo que qualquer palavra possa ser representada, mesmo que nunca tenha aparecido no corpus original.

4. **Decode (reconstrução):**
   As sequências de subpalavras são combinadas para reconstruir o texto original, preservando a estrutura das palavras.

Esse mecanismo permite que modelos de linguagem equilibrem **tamanho de vocabulário**, **expressividade** e **generalização**, sendo a base de tokenizadores como os usados em GPT, BERT e outros LLMs.

**Tokenizadores reais:**

* **Open AI** https://platform.openai.com/tokenizer
* **Tiktokenizer** https://tiktokenizer.vercel.app/

## Implementação de um BPE

In [1]:
import re
from collections import Counter, defaultdict
from collections.abc import Iterable
from dataclasses import dataclass


@dataclass
class BPEResult:
    """Armazena resultados do treinamento do BPE (didático)."""

    vocab: dict[
        tuple[str, ...], int
    ]  # representação do corpus como "palavra tokenizada" -> contagem
    merges: list[tuple[str, str]]  # lista ordenada de merges aprendidos
    token_to_id: dict[str, int]  # vocabulário final (tokens) -> id
    id_to_token: dict[int, str]  # id -> token


class BytePairEncoderDecoder:
    """
    Byte Pair Encoding (BPE) do zero, com passo a passo (didático).

    Ideia central (BPE clássico para subpalavras):
    - Representamos cada palavra como caracteres + marcador de fim de palavra (</w>)
    - Contamos pares adjacentes mais frequentes (bigramas de símbolos)
    - Iterativamente "fundimos" (merge) o par mais frequente em um novo símbolo
    - Guardamos a sequência de merges para tokenizar (encode) textos novos

    Observação:
    - Este é o BPE "clássico" baseado em palavras, suficiente para entender o mecanismo.
    - Em tokenizadores modernos (ex.: GPT-2), há detalhes adicionais (bytes, etc.),
      mas o coração do BPE (aprender merges frequentes) é o mesmo.
    """

    def __init__(self, end_of_word: str = "</w>") -> None:
        """
        Parâmetros:
        ----------
        end_of_word : str, default="</w>"
            Marcador de fim de palavra (ajuda o BPE a não "colar" palavras entre si).
        """
        if not isinstance(end_of_word, str) or not end_of_word:
            raise ValueError("`end_of_word` deve ser uma string não vazia.")
        self.end_of_word = end_of_word

        # aprendidos após treino
        self.merges: list[tuple[str, str]] = []
        self.token_to_id: dict[str, int] = {}
        self.id_to_token: dict[int, str] = {}

    # ======================================================
    # Utilitários de pré-processamento (didáticos e simples)
    # ======================================================
    @staticmethod
    def _basic_word_tokenize(text: str) -> list[str]:
        """
        Tokeniza em "palavras" de forma simples (didática).
        - Mantém letras e números como parte das palavras
        - Trata pontuação como separador

        Ex.: "O gato, sobe." -> ["O", "gato", "sobe"]
        """
        if not isinstance(text, str):
            raise TypeError("O parâmetro `text` deve ser uma string.")
        words = re.findall(r"[A-Za-zÀ-ÿ0-9]+", text)
        return words

    def _word_to_symbols(self, word: str) -> tuple[str, ...]:
        """
        Converte uma palavra em uma tupla de símbolos (caracteres + </w>).
        """
        if not isinstance(word, str) or not word:
            raise ValueError("`word` deve ser uma string não vazia.")
        return tuple(list(word) + [self.end_of_word])

    # =========================================
    # PASSO A PASSO: construção do "vocab" BPE
    # =========================================
    def build_initial_vocab(
        self, corpus: Iterable[str], verbose: bool = True
    ) -> dict[tuple[str, ...], int]:
        """
        Constrói o vocabulário inicial do BPE:
        - Cada palavra vira uma sequência de caracteres + </w>
        - Contabilizamos frequência de cada "palavra tokenizada"

        Retorno:
        -------
        dict[tuple[str, ...], int]
            Mapeia (símbolos da palavra) -> contagem
        """
        if corpus is None:
            raise TypeError("`corpus` não pode ser None.")

        corpus = list(corpus)
        if not corpus:
            raise ValueError("`corpus` precisa conter ao menos 1 texto/frase.")

        vocab: dict[tuple[str, ...], int] = defaultdict(int)

        if verbose:
            print("=== PASSO 1: Corpus bruto ===")
            for i, line in enumerate(corpus, start=1):
                print(f"{i}. {line}")
            print()

        if verbose:
            print("=== PASSO 2: Quebrar corpus em palavras (tokenização simples) ===")

        all_words: list[str] = []
        for line in corpus:
            words = self._basic_word_tokenize(line)
            all_words.extend(words)

        if verbose:
            print("Palavras extraídas:", all_words)
            print()

        if verbose:
            print("=== PASSO 3: Vocabulário inicial (caracteres + </w>) ===")

        for w in all_words:
            sym = self._word_to_symbols(w)
            vocab[sym] += 1

        if verbose:
            for sym, cnt in sorted(vocab.items(), key=lambda x: (-x[1], x[0])):
                print(f"{' '.join(sym)}  ->  {cnt}")
            print()

        return dict(vocab)

    # =========================================
    # PASSO A PASSO: contar pares e aplicar merge
    # =========================================
    @staticmethod
    def _get_pair_frequencies(
        vocab: dict[tuple[str, ...], int],
    ) -> Counter[tuple[str, str]]:
        """
        Conta a frequência de pares adjacentes em todo o vocabulário.
        """
        pairs: Counter[tuple[str, str]] = Counter()
        for word_symbols, freq in vocab.items():
            # ex.: ("l","o","w","</w>") -> pares: ("l","o"), ("o","w"), ("w","</w>")
            for i in range(len(word_symbols) - 1):
                pairs[(word_symbols[i], word_symbols[i + 1])] += freq
        return pairs

    @staticmethod
    def _merge_pair_in_word(
        word_symbols: tuple[str, ...], pair: tuple[str, str]
    ) -> tuple[str, ...]:
        """
        Aplica um merge (a,b) dentro de uma palavra tokenizada:
        substitui ocorrências adjacentes de a b por "ab".
        """
        a, b = pair
        merged_symbol = a + b

        new_symbols: list[str] = []
        i = 0
        while i < len(word_symbols):
            # se encontramos a,b adjacentes, funde
            if (
                i < len(word_symbols) - 1
                and word_symbols[i] == a
                and word_symbols[i + 1] == b
            ):
                new_symbols.append(merged_symbol)
                i += 2
            else:
                new_symbols.append(word_symbols[i])
                i += 1

        return tuple(new_symbols)

    def train(
        self,
        corpus: Iterable[str],
        num_merges: int = 30,
        verbose: bool = True,
        top_pairs_to_show: int = 10,
    ) -> BPEResult:
        """
        Treina (aprende) merges BPE a partir do corpus.

        Parâmetros:
        ----------
        corpus : Iterable[str]
            Textos/frases para treinar o BPE.
        num_merges : int, default=30
            Número de merges (iterações) a aprender.
        verbose : bool, default=True
            Se True, imprime passo a passo.
        top_pairs_to_show : int, default=10
            Quantos pares mais frequentes exibir por iteração (didático).

        Retorno:
        -------
        BPEResult
            Estruturas do BPE treinado (vocab final, merges e mapeamentos).
        """
        if not isinstance(num_merges, int) or num_merges <= 0:
            raise ValueError("`num_merges` deve ser um inteiro > 0.")

        vocab = self.build_initial_vocab(corpus, verbose=verbose)

        self.merges = []

        for step in range(1, num_merges + 1):
            pairs = self._get_pair_frequencies(vocab)

            if not pairs:
                if verbose:
                    print("Nenhum par restante para fundir. Encerrando.")
                break

            best_pair, best_freq = pairs.most_common(1)[0]

            if verbose:
                print(
                    f"=== PASSO 4.{step}: Contagem de pares (top {top_pairs_to_show}) ==="
                )
                for p, f in pairs.most_common(top_pairs_to_show):
                    print(f"{p} -> {f}")
                print()
                print(f"=== PASSO 5.{step}: Melhor par para merge ===")
                print(f"Par escolhido: {best_pair} (freq={best_freq})")
                print(f"Novo símbolo: '{best_pair[0] + best_pair[1]}'")
                print()

            # aplica merge ao vocab inteiro
            new_vocab: dict[tuple[str, ...], int] = defaultdict(int)
            for word_symbols, freq in vocab.items():
                merged = self._merge_pair_in_word(word_symbols, best_pair)
                new_vocab[merged] += freq

            vocab = dict(new_vocab)
            self.merges.append(best_pair)

            if verbose:
                print(f"=== PASSO 6.{step}: Vocabulário após merge ===")
                # imprime um resumo do vocab (palavras tokenizadas + contagem)
                for sym, cnt in sorted(vocab.items(), key=lambda x: (-x[1], x[0]))[:15]:
                    print(f"{' '.join(sym)}  ->  {cnt}")
                if len(vocab) > 15:
                    print("... (mostrando apenas as 15 primeiras entradas)")
                print()

        # Montar "vocab final" de tokens: todos os símbolos que aparecem no vocab após merges
        final_tokens: set[str] = set()
        for word_symbols in vocab.keys():
            final_tokens.update(word_symbols)

        # Ordem estável (didática): ordena alfabeticamente
        final_tokens_sorted = sorted(final_tokens)
        self.token_to_id = {tok: i for i, tok in enumerate(final_tokens_sorted)}
        self.id_to_token = {i: tok for tok, i in self.token_to_id.items()}

        if verbose:
            print("=== PASSO 7: Tokens finais (vocabulário de subpalavras) ===")
            print(final_tokens_sorted)
            print()
            print("=== PASSO 8: token_to_id (amostra) ===")
            for tok in final_tokens_sorted[:30]:
                print(f"{tok:10s} -> {self.token_to_id[tok]}")
            if len(final_tokens_sorted) > 30:
                print("... (mostrando apenas 30 tokens)")
            print()

        return BPEResult(
            vocab=vocab,
            merges=self.merges.copy(),
            token_to_id=self.token_to_id.copy(),
            id_to_token=self.id_to_token.copy(),
        )

    # =========================================
    # Encode / Decode com passo a passo
    # =========================================
    def _apply_merges_to_symbols(
        self, symbols: tuple[str, ...], verbose: bool = True
    ) -> tuple[str, ...]:
        """
        Aplica os merges aprendidos, na ordem, a uma sequência de símbolos.
        """
        if verbose:
            print("Símbolos iniciais:", symbols)

        for i, pair in enumerate(self.merges, start=1):
            new_symbols = self._merge_pair_in_word(symbols, pair)
            if new_symbols != symbols and verbose:
                print(f"Merge {i}: {pair} -> '{pair[0] + pair[1]}'")
                print("Antes:", symbols)
                print("Depois:", new_symbols)
                print()
            symbols = new_symbols

        if verbose:
            print("Símbolos finais (após merges):", symbols)
            print()

        return symbols

    def encode(self, text: str, verbose: bool = True) -> list[int]:
        """
        Tokeniza (BPE) e converte em IDs.

        Retorno:
        -------
        list[int]
            IDs dos tokens BPE.
        """
        if not self.token_to_id or not self.merges:
            raise ValueError("BPE não treinado. Execute `train()` antes de `encode()`.")

        if verbose:
            print("=== ENCODE (BPE) ===")
            print("Texto:", text)
            print()

        words = self._basic_word_tokenize(text)
        if verbose:
            print("Palavras:", words)
            print()

        all_ids: list[int] = []
        for w in words:
            if verbose:
                print(f"--- Palavra: '{w}' ---")
            symbols = self._word_to_symbols(w)
            final_symbols = self._apply_merges_to_symbols(symbols, verbose=verbose)

            # converte símbolos finais em ids (todos devem existir no vocab final)
            ids = []
            for s in final_symbols:
                if s not in self.token_to_id:
                    raise ValueError(
                        f"Token '{s}' não existe no vocabulário final. (Isso não deveria ocorrer neste BPE.)"
                    )
                ids.append(self.token_to_id[s])

            if verbose:
                print("Tokens BPE:", final_symbols)
                print("IDs:", ids)
                print()

            all_ids.extend(ids)

        if verbose:
            print("=== SEQUÊNCIA FINAL DE IDS ===")
            print(all_ids)
            print()

        return all_ids

    def decode(self, ids: list[int], verbose: bool = True) -> str:
        """
        Converte IDs -> tokens e reconstrói texto.

        Regra:
        - Junta tokens
        - Substitui </w> por espaço de palavra
        """
        if not isinstance(ids, list) or any(not isinstance(i, int) for i in ids):
            raise TypeError("`ids` deve ser uma lista de inteiros (list[int]).")
        if not self.id_to_token:
            raise ValueError("BPE não treinado. Execute `train()` antes de `decode()`.")

        if verbose:
            print("=== DECODE (BPE) ===")
            print("IDs:", ids)
            print()

        tokens = []
        for i in ids:
            if i not in self.id_to_token:
                raise ValueError(f"ID inválido: {i}")
            tokens.append(self.id_to_token[i])

        if verbose:
            print("Tokens:", tokens)
            print()

        # Reconstrução:
        # - removemos o marcador </w> trocando por espaço
        # - depois fazemos strip para remover espaço final
        text = ""
        for t in tokens:
            if t == self.end_of_word:
                text += " "
            else:
                text += t

        text = text.strip()

        if verbose:
            print("Texto reconstruído:", text)
            print()

        return text

## Geração do corpus

In [2]:
corpus = [
    "O gato sobe no tapete.",
    "O cachorro sobe na mesa.",
    "A aranha desce a parede.",
    "O gato desce da mesa.",
]

bpe = BytePairEncoderDecoder(end_of_word="</w>")

# Treinar com poucas merges para ficar bem visível no passo a passo
resultado = bpe.train(corpus=corpus, num_merges=10, verbose=True, top_pairs_to_show=10)

=== PASSO 1: Corpus bruto ===
1. O gato sobe no tapete.
2. O cachorro sobe na mesa.
3. A aranha desce a parede.
4. O gato desce da mesa.

=== PASSO 2: Quebrar corpus em palavras (tokenização simples) ===
Palavras extraídas: ['O', 'gato', 'sobe', 'no', 'tapete', 'O', 'cachorro', 'sobe', 'na', 'mesa', 'A', 'aranha', 'desce', 'a', 'parede', 'O', 'gato', 'desce', 'da', 'mesa']

=== PASSO 3: Vocabulário inicial (caracteres + </w>) ===
O </w>  ->  3
d e s c e </w>  ->  2
g a t o </w>  ->  2
m e s a </w>  ->  2
s o b e </w>  ->  2
A </w>  ->  1
a </w>  ->  1
a r a n h a </w>  ->  1
c a c h o r r o </w>  ->  1
d a </w>  ->  1
n a </w>  ->  1
n o </w>  ->  1
p a r e d e </w>  ->  1
t a p e t e </w>  ->  1

=== PASSO 4.1: Contagem de pares (top 10) ===
('e', '</w>') -> 6
('a', '</w>') -> 6
('o', '</w>') -> 4
('e', 's') -> 4
('O', '</w>') -> 3
('d', 'e') -> 3
('g', 'a') -> 2
('a', 't') -> 2
('t', 'o') -> 2
('s', 'o') -> 2

=== PASSO 5.1: Melhor par para merge ===
Par escolhido: ('e', '</w>') (fre

## Exemplo de uso do encode (uma frase ainda não utilizada)

In [3]:
# Encode: veja como palavras viram subpalavras/bytes (aqui: símbolos BPE)
texto_novo = "A aranha sobe no gato."
ids = bpe.encode(texto_novo, verbose=True)

=== ENCODE (BPE) ===
Texto: A aranha sobe no gato.

Palavras: ['A', 'aranha', 'sobe', 'no', 'gato']

--- Palavra: 'A' ---
Símbolos iniciais: ('A', '</w>')
Símbolos finais (após merges): ('A', '</w>')

Tokens BPE: ('A', '</w>')
IDs: [1, 0]

--- Palavra: 'aranha' ---
Símbolos iniciais: ('a', 'r', 'a', 'n', 'h', 'a', '</w>')
Merge 2: ('a', '</w>') -> 'a</w>'
Antes: ('a', 'r', 'a', 'n', 'h', 'a', '</w>')
Depois: ('a', 'r', 'a', 'n', 'h', 'a</w>')

Símbolos finais (após merges): ('a', 'r', 'a', 'n', 'h', 'a</w>')

Tokens BPE: ('a', 'r', 'a', 'n', 'h', 'a</w>')
IDs: [3, 17, 3, 13, 11, 4]

--- Palavra: 'sobe' ---
Símbolos iniciais: ('s', 'o', 'b', 'e', '</w>')
Merge 1: ('e', '</w>') -> 'e</w>'
Antes: ('s', 'o', 'b', 'e', '</w>')
Depois: ('s', 'o', 'b', 'e</w>')

Merge 9: ('s', 'o') -> 'so'
Antes: ('s', 'o', 'b', 'e</w>')
Depois: ('so', 'b', 'e</w>')

Merge 10: ('so', 'b') -> 'sob'
Antes: ('so', 'b', 'e</w>')
Depois: ('sob', 'e</w>')

Símbolos finais (após merges): ('sob', 'e</w>')

Tokens BPE

## Exemplo de uso do decode (Uma frase ainda não utilizada)

In [4]:
# Decode: volta ao texto
texto_reconstruido = bpe.decode(ids, verbose=True)

=== DECODE (BPE) ===
IDs: [1, 0, 3, 17, 3, 13, 11, 4, 18, 8, 13, 15, 10]

Tokens: ['A', '</w>', 'a', 'r', 'a', 'n', 'h', 'a</w>', 'sob', 'e</w>', 'n', 'o</w>', 'gato</w>']

Texto reconstruído: A aranha</w>sobe</w>no</w>gato</w>



## Este método é naturalmente robusto a palavras desconhecidas

In [5]:
# Encode: veja como palavras viram subpalavras/bytes (aqui: símbolos BPE)
texto_novo = "A aranha sobe no rato."
ids = bpe.encode(texto_novo, verbose=True)

=== ENCODE (BPE) ===
Texto: A aranha sobe no rato.

Palavras: ['A', 'aranha', 'sobe', 'no', 'rato']

--- Palavra: 'A' ---
Símbolos iniciais: ('A', '</w>')
Símbolos finais (após merges): ('A', '</w>')

Tokens BPE: ('A', '</w>')
IDs: [1, 0]

--- Palavra: 'aranha' ---
Símbolos iniciais: ('a', 'r', 'a', 'n', 'h', 'a', '</w>')
Merge 2: ('a', '</w>') -> 'a</w>'
Antes: ('a', 'r', 'a', 'n', 'h', 'a', '</w>')
Depois: ('a', 'r', 'a', 'n', 'h', 'a</w>')

Símbolos finais (após merges): ('a', 'r', 'a', 'n', 'h', 'a</w>')

Tokens BPE: ('a', 'r', 'a', 'n', 'h', 'a</w>')
IDs: [3, 17, 3, 13, 11, 4]

--- Palavra: 'sobe' ---
Símbolos iniciais: ('s', 'o', 'b', 'e', '</w>')
Merge 1: ('e', '</w>') -> 'e</w>'
Antes: ('s', 'o', 'b', 'e', '</w>')
Depois: ('s', 'o', 'b', 'e</w>')

Merge 9: ('s', 'o') -> 'so'
Antes: ('s', 'o', 'b', 'e</w>')
Depois: ('so', 'b', 'e</w>')

Merge 10: ('so', 'b') -> 'sob'
Antes: ('so', 'b', 'e</w>')
Depois: ('sob', 'e</w>')

Símbolos finais (após merges): ('sob', 'e</w>')

Tokens BPE

## Decode com a palavra desconhecida

In [6]:
# Decode: volta ao texto
texto_reconstruido = bpe.decode(ids, verbose=True)

=== DECODE (BPE) ===
IDs: [1, 0, 3, 17, 3, 13, 11, 4, 18, 8, 13, 15, 17, 3, 19, 15]

Tokens: ['A', '</w>', 'a', 'r', 'a', 'n', 'h', 'a</w>', 'sob', 'e</w>', 'n', 'o</w>', 'r', 'a', 't', 'o</w>']

Texto reconstruído: A aranha</w>sobe</w>no</w>rato</w>



## O número de iterações influencia nos tokens gerados

### Iterações = 1

In [7]:
corpus = [
    "O gato sobe no tapete.",
    "O cachorro sobe na mesa.",
    "A aranha desce a parede.",
    "O gato desce da mesa.",
]

bpe = BytePairEncoderDecoder(end_of_word="</w>")
resultado = bpe.train(corpus=corpus, num_merges=1, verbose=False)
resultado.token_to_id

{'</w>': 0,
 'A': 1,
 'O': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'e</w>': 8,
 'g': 9,
 'h': 10,
 'm': 11,
 'n': 12,
 'o': 13,
 'p': 14,
 'r': 15,
 's': 16,
 't': 17}

In [8]:
texto_novo = "O gato sobe no tapete O cachorro sobe na mesa"
ids = bpe.encode(texto_novo, verbose=False)
print(f"{ids=}, {len(ids)=}")

ids=[2, 0, 9, 3, 17, 13, 0, 16, 13, 4, 8, 12, 13, 0, 17, 3, 14, 7, 17, 8, 2, 0, 5, 3, 5, 10, 13, 15, 15, 13, 0, 16, 13, 4, 8, 12, 3, 0, 11, 7, 16, 3, 0], len(ids)=43


### Iterações = 20

In [9]:
bpe = BytePairEncoderDecoder(end_of_word="</w>")
resultado = bpe.train(corpus=corpus, num_merges=20, verbose=False)
resultado.token_to_id

{'</w>': 0,
 'A': 1,
 'O</w>': 2,
 'a': 3,
 'a</w>': 4,
 'ar': 5,
 'c': 6,
 'd': 7,
 'desce</w>': 8,
 'e': 9,
 'e</w>': 10,
 'gato</w>': 11,
 'h': 12,
 'mesa</w>': 13,
 'n': 14,
 'no</w>': 15,
 'o': 16,
 'o</w>': 17,
 'p': 18,
 'r': 19,
 'sobe</w>': 20,
 't': 21,
 'tap': 22}

In [10]:
texto_novo = "O gato sobe no tapete O cachorro sobe na mesa"
ids = bpe.encode(texto_novo, verbose=False)
print(f"{ids=}, {len(ids)=}")

ids=[2, 11, 20, 15, 22, 9, 21, 10, 2, 6, 3, 6, 12, 16, 19, 19, 17, 20, 14, 4, 13], len(ids)=21


### Iterações = 40

In [11]:
bpe = BytePairEncoderDecoder(end_of_word="</w>")
resultado = bpe.train(corpus=corpus, num_merges=40, verbose=False)
resultado.token_to_id

{'A</w>': 0,
 'O</w>': 1,
 'a</w>': 2,
 'aranha</w>': 3,
 'cachorro</w>': 4,
 'd': 5,
 'desce</w>': 6,
 'gato</w>': 7,
 'mesa</w>': 8,
 'na</w>': 9,
 'no</w>': 10,
 'parede</w>': 11,
 'sobe</w>': 12,
 'tapete</w>': 13}

In [12]:
texto_novo = "O gato sobe no tapete O cachorro sobe na mesa"
ids = bpe.encode(texto_novo, verbose=False)
print(f"{ids=}, {len(ids)=}")

ids=[1, 7, 12, 10, 13, 1, 4, 12, 9, 8], len(ids)=10
